<a href="https://colab.research.google.com/github/Renuka-cell/IBM_project_file/blob/main/Big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [9]:
#!wget -q http://apache.mirrors.lucidnetworks.net/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!wget https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

--2025-08-07 08:37:17--  https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388341449 (370M) [application/x-gzip]
Saving to: ‘spark-3.4.1-bin-hadoop3.tgz.1’

spark-3.4.1-bin-had 100%[===================>] 370.35M   277KB/s    in 25m 11s 

2025-08-07 09:02:29 (251 KB/s) - ‘spark-3.4.1-bin-hadoop3.tgz.1’ saved [388341449/388341449]



In [12]:
!tar xf spark-3.4.1-bin-hadoop3.tgz

In [13]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

In [16]:
!pip install -q findspark

In [17]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [19]:
from google.colab import files
upload=files.upload()

df=spark.read.csv("BigBasket Products.csv",header=True,inferSchema=True)
df.show(5)
df.printSchema()

Saving BigBasket Products.csv to BigBasket Products (1).csv
+--------------------+--------------------+--------------------+--------------------+-----------------+----------+------------+--------------------+------+--------------------+
|               index|             product|            category|        sub_category|            brand|sale_price|market_price|                type|rating|         description|
+--------------------+--------------------+--------------------+--------------------+-----------------+----------+------------+--------------------+------+--------------------+
|                   1|Garlic Oil - Vege...|    Beauty & Hygiene|           Hair Care|Sri Sri Ayurveda |       220|         220|    Hair Oil & Serum|   4.1|This Product cont...|
|                   2|Water Bottle - Or...|Kitchen, Garden &...|Storage & Accesso...|       Mastercook|       180|         180|Water & Fridge Bo...|   2.3|Each product is m...|
|                   3|Brass Angle Deep ...|Cleaning & H

In [20]:
from pyspark.sql.functions import col

df_cleaned= df.dropna()
df_cleaned.select([col(c).isNull().alias(c) for c in df_cleaned.columns]).show()

+-----+-------+--------+------------+-----+----------+------------+-----+------+-----------+
|index|product|category|sub_category|brand|sale_price|market_price| type|rating|description|
+-----+-------+--------+------------+-----+----------+------------+-----+------+-----------+
|false|  false|   false|       false|false|     false|       false|false| false|      false|
|false|  false|   false|       false|false|     false|       false|false| false|      false|
|false|  false|   false|       false|false|     false|       false|false| false|      false|
|false|  false|   false|       false|false|     false|       false|false| false|      false|
|false|  false|   false|       false|false|     false|       false|false| false|      false|
|false|  false|   false|       false|false|     false|       false|false| false|      false|
|false|  false|   false|       false|false|     false|       false|false| false|      false|
|false|  false|   false|       false|false|     false|       false|fal

In [22]:
from pyspark.sql.functions import rand, expr
from pyspark.sql.functions import current_timestamp

df_augmented = df_cleaned \
    .withColumn("user_id", (rand()*1000).cast("int")) \
    .withColumn("quantity", (rand()*5 + 1).cast("int")) \
    .withColumn("timestamp", expr("current_timestamp()")) \
    .withColumn("payment_method", expr("CASE WHEN rand() < 0.3 THEN 'Card' WHEN rand() < 0.6 THEN 'Cash' ELSE 'UPI' END")) \
    .withColumn("location", expr("CASE WHEN rand() < 0.5 THEN 'Mumbai' ELSE 'Bangalore' END"))

df_augmented.show(5)

+-----+--------------------+--------------------+--------------------+-----------------+----------+------------+--------------------+------+--------------------+-------+--------+--------------------+--------------+---------+
|index|             product|            category|        sub_category|            brand|sale_price|market_price|                type|rating|         description|user_id|quantity|           timestamp|payment_method| location|
+-----+--------------------+--------------------+--------------------+-----------------+----------+------------+--------------------+------+--------------------+-------+--------+--------------------+--------------+---------+
|    1|Garlic Oil - Vege...|    Beauty & Hygiene|           Hair Care|Sri Sri Ayurveda |       220|         220|    Hair Oil & Serum|   4.1|This Product cont...|    831|       3|2025-08-07 09:28:...|          Card|   Mumbai|
|    2|Water Bottle - Or...|Kitchen, Garden &...|Storage & Accesso...|       Mastercook|       180| 

In [28]:
from pyspark.sql.functions import sum

df_augmented = df_augmented.withColumn("revenue", col("sale_price") * col("quantity"))
df_augmented.agg(sum("revenue")).show()

df_augmented.show(5)

+--------------------+
|        sum(revenue)|
+--------------------+
|1.5082174110000005E7|
+--------------------+

+-----+--------------------+--------------------+--------------------+-----------------+----------+------------+--------------------+------+--------------------+-------+--------+--------------------+--------------+---------+-------+
|index|             product|            category|        sub_category|            brand|sale_price|market_price|                type|rating|         description|user_id|quantity|           timestamp|payment_method| location|revenue|
+-----+--------------------+--------------------+--------------------+-----------------+----------+------------+--------------------+------+--------------------+-------+--------+--------------------+--------------+---------+-------+
|    1|Garlic Oil - Vege...|    Beauty & Hygiene|           Hair Care|Sri Sri Ayurveda |       220|         220|    Hair Oil & Serum|   4.1|This Product cont...|    831|       3|2025-08

In [26]:
from pyspark.sql.functions import count

df_augmented.groupBy("product").agg(count("*").alias("purchase_count")) \
    .orderBy("purchase_count", ascending=False).show()

df_augmented.show(5)

+--------------------+--------------+
|             product|purchase_count|
+--------------------+--------------+
|Turmeric Powder/A...|            23|
|          Soft Drink|            12|
|      Cow Ghee/Tuppa|            11|
|          Ghee/Tuppa|            11|
|Colorsilk Hair Co...|            10|
|  Powder - Coriander|            10|
|Casting Creme Glo...|            10|
|    Masala - Chicken|             9|
|      Hand Sanitizer|             9|
|Extra Virgin Oliv...|             9|
|Anti Dandruff Sha...|             9|
|                 Tea|             8|
|Excellence Creme ...|             8|
|  Olive Oil - Pomace|             8|
|    Coriander Powder|             8|
|Peanut Butter - C...|             8|
|Color Naturals - ...|             8|
|Extra Light Olive...|             8|
|          phosphorus|             7|
|Pantry/Cookie/Sna...|             7|
+--------------------+--------------+
only showing top 20 rows

+-----+--------------------+--------------------+-------------

In [29]:
from pyspark.sql.functions import hour

df_augmented.withColumn("hour", hour("timestamp")).groupBy("hour").count().show()

+----+-----+
|hour|count|
+----+-----+
|   9|19044|
+----+-----+



In [30]:
df_augmented.groupBy("location").agg(sum("revenue").alias("total_revenue")) \
    .orderBy("total_revenue", ascending=False).show()


+---------+-----------------+
| location|    total_revenue|
+---------+-----------------+
|Bangalore|7591308.260000002|
|   Mumbai|7490865.850000004|
+---------+-----------------+



In [31]:
customer_df = df_augmented.groupBy("user_id").agg(
    sum("revenue").alias("total_spent"),
    count("product").alias("frequency"),
    (sum("revenue") / count("product")).alias("avg_basket_size")
)
customer_df.show()

+-------+------------------+---------+------------------+
|user_id|       total_spent|frequency|   avg_basket_size|
+-------+------------------+---------+------------------+
|    496|          25787.46|       19|1357.2347368421051|
|    471|          17091.75|       23| 743.1195652173913|
|    148|           19232.0|       22| 874.1818181818181|
|    833|           13305.1|       15| 887.0066666666667|
|    463|11852.189999999999|       17| 697.1876470588235|
|    858|          25007.64|       20|          1250.382|
|    623|          14693.55|       19| 773.3447368421052|
|    897|          17937.73|       20|          896.8865|
|    243|14067.009999999998|       19|  740.368947368421|
|    540|          19906.46|       15|1327.0973333333334|
|    392|           9580.24|       20|           479.012|
|    737|           7241.65|       14| 517.2607142857142|
|    516|           11271.0|       22| 512.3181818181819|
|     31|          13808.92|       15| 920.5946666666666|
|    580|     